# QM point-particle in exetrnal potential 
___
Daniel Cierpinsky 641249, Jann Ole Neitzel 590359

## Conceptual approach & code implementation

All methods utilized in this project are based on the concept of lattice discretization, which makes the computational calculation of the Schrödinger equation possible in the first place. Specifically it is required to assign discrete numerical values or operations to quantum-mechanical concepts such as the wave function $\Psi$ or Hamiltonian operator $\hat{H}$.

In this case, arrays will be used to represent them. Independently of the problem to be described, during discretization the resolution needs to be defined. Here we define the integer $N$ to the size of the array and $D$ the dimension.
Under consideration of the fact multidimensional arrays are to be expected, we chose to  define the arrays as `numpy.ndarray`. This has the added benefit of being able to use powerful `numpy` array operations such as `numpy.vdot` and `numpy.roll` directly on the arrays without the need to flatten and reshape them.

Operators can not be represented as arrays, but are instead be converted to discrete array operations. For this purpose all funcions have been designed with special attention so that they can take an `numpy.ndarray` as input and apply the operations only on the array as whole (not element-wise). This not only improves efficiency by using pre-built `numpy` functions (compared to element-wise operations), but also makes the code more readable (to the point of recognizing the original operation).

Example:

In [2]:
from functions import *
def expectation_energy(psi:np.ndarray)->float:
    """Calculates expectation value of energy for wavefunction psi."""
    return np.vdot(psi, hamiltonian_function(psi)).real

## Considerations
### Units
As this project is based on numerical calculation, the units chosed during lattice discretization have to be taken into consideration. 
A common approach is to work exclusively with unitless numbers, which results in a larger upfront effort during programming but results in compatibilty of results with any unit system. 
To discretize functions and operators to unitless values, following terms have been chosen to describe the system:\
$L$ : physical extent of the space in which the system is defined. Increasing this value increases the real space in which the calculations will be executed,\
$N$ : unitless number of lattice points (per dimension) in which the system is subdivided, \
$a = L/N$ : real space distance between lattice points,\  
$D$ : number of dimensions the system is defined in,\
$r$ : for this specific problem, this is the parameter which describes extension of the potential,\
$\omega$ : frequency term which defines the potential,
\
$\varepsilon = \frac{a}{r}$ : unitless term which describes lattice resolution relative to extension of the potential,\
$\mu = \frac{m \omega r^{2}}{\hbar}$ : dimensionless term originating from the definition of the kinetic energy. Used to convert $E_{kin}$ and $\hat{p}$ to unitless values.

The terms $\mu, \varepsilon, N$ are used during the implementation of operators into functions to ensure the results will be unitless. \
As an example, below is the calculation to convert the result of the impulse operator applied to a wavefunction $p\ket{\Psi}$ into unitless values $\hat{p}\ket{\hat{\Psi}}$:

$\hat{p} = \frac{p}{mass\cdot length /time} = \frac{p}{m \cdot r \cdot \omega}$\
\
$\implies  \hat{p}\ket{\hat{\Psi}} = \frac{a^{D/2}}{m \omega r} p\ket{\Psi} = \frac{a^{D/2}}{m \omega r}(-i\hbar)\nabla\Psi$\
\
$ = \frac{-i\hbar a^{D/2}}{m \omega r}\cdot\frac{\sum\limits_{i=0}^D \hat{\Psi}(n_{i}+e_{i})- \hat{\Psi}(n_{i}-e_{i})}{2a}\cdot a^{-D/2} $\
\
$ = -\frac{i}{2}\cdot\frac{\hbar}{m \omega r}\cdot\frac{1}{r}\cdot\frac{r}{1}\cdot\frac{1}{a}\sum\limits_{i=0}^D \hat{\Psi}(n_{i}+e_{i})- \hat{\Psi}(n_{i}-e_{i})$\
\
$ = -\frac{i}{2}\cdot\frac{1}{\mu}\cdot\frac{1}{\varepsilon}\sum\limits_{i=0}^D \hat{\Psi}(n_{i}+e_{i})- \hat{\Psi}(n_{i}-e_{i})$


### Periodicity

The derivative as defined in the lattice discretization requires to acces an arrays neighbouring entries. This poses a problem since we are using finite arrays. One approach to overcome this issue is defining the origin of the system in the center of the array. Since the potential $V$ is symmetrical, the opposite ends of the array will have the same values `array[-N/2] = array[N/2]`. \
The discretized derivative operation $\hat{\Psi}(n_{i}+e_{i})- \hat{\Psi}(n_{i}-e_{i})$ can therefore be easily implemented using the `numpy.roll` which shifts the entries of the array in the  chosen direction $i$.



### Representation of results

Representing the eigenfunction of the hamiltonian requires two further steps to ensure unitless-ness:

$x$ [units of $a$] $\rightarrow x\cdot \varepsilon$ [unitless] \
$|\Psi|^{2}$ [units of $a^{-D}$] $\rightarrow |\Psi|^{2}/ \varepsilon^{D}$

Example plots can be found in `plotting.ipynb`


## Results & Discussion
(please refer to `ground_state.ipynb` for the code and plots of the ground state)

The ground state corresponds to the lowest eigenvalue and eigenfunction pair of the Hamilton operator.
Taking advantage of the fact the hamilton operator behaves like a positive definite matrix we shall call $A$, we can use the power method to determine the largest eigenvalue and eigenvector of $A$.
The largest eigenvalue of the matrix $A^{-1}$ is exatly the smallest eigenvalue of the matrix $A$. Conveniently, we can use the conjugate gradient method to determine the inverse of the matrix $A^{-1}$ applied (for example vector product) on a vector.
We can take advantage of the fact the power method requires just such an operation as the conjugate gradient to iterate, we can make it calculate the largest eigenvalue of $A^{-1}$.
In both cases, only the result of the matrix operation is required, which means we can implement the hamilton operator any way we want without the need of representing it as a matrix. So, if we use the conjugate gradient on the hamilton operator function and let this result be computed by the power method, we obtain the lowest eigenvalue and eigenfunction of the Hamilton operator.

To calculate the ground state of the hamiltonian we first use a discretized lattice in 1 dimension. The resulting eigenfunctions are sometimes symmetrical and antisymmetrical, whereby the amplitudes of anti-nodes vary for each calculation. The extremes of the eigenfunction are exactly at $x = \pm r$, which is the position of the minima of the potential $V$. This coincides with our expectations. We expect the symmetric wavefunction to be the ground state for given potential. Both the fact that some of the results are antisymmetric and without fixed amplitude suggests some form of inconsistenticy in  the iterative process producing the results. One hypothesis would be that during the iterations, the large tolerance leads to the determination of the next highest state above the ground state, which would exactly be the antisymmetrical function we obtained.


Next we calculated the ground state of the hamiltoninan for a discrete lattice in 2 dimensions. We notice the very long computational time required to determine the eigenvalue and eigenfunction for the given parameters (same as 1D except for dimension).
The obtained eigenfunction is a radially symmetric wavefunction with maxima at $x =\pm r$. As such the determined eigenfunction fullfills our expectations for the 2-dimensional case.


Further we want to discuss the performance of lattice discretiation for the infinite volume limit $L \rightarrow \infty$ and the continuum limit $a \rightarrow 0$.

For the infinite volume limit we observe that for equal number of lattice points N, the spatial resolution of the eigenfunction is reduced. This results from the fact the same number of lattice points describe a growing volume interval. This is not favorable for the problem we are trying to describe, as the infinite volume limit potential approaches infinity: $\lim_{{L \to \infty}} V(L) = \infty$. The wave functions is therefore $ \approx 0$ for most of the volume we describe with the lattice discretization.

On the other hand, the continuum limit has the opposite effect. The fixed number of lattice points will describe an increasingly smaller volume interval. This is beneficial for the resolution of the calculated eigenfunction. One important consideration for the continuum limit is that eventually the lattice will not cover the full relevant range of the potential $-1<\frac{x}{r}<1$. This will result in an inaccurate calculation of the eigenfunction, as the potential is not fully described anymore.
